In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

In [3]:
mnist = input_data.read_data_sets('MNIST-data' ,  one_hot = True)
trainimg  = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [23]:
# 构建 network

n_input = 784
n_output = 10
_keepratio = 0.75

weights = {
    
    'wc1' : tf.Variable(tf.random_normal( [ 3, 3, 1, 64 ], stddev = 0.1)),
    'wc2' : tf.Variable(tf.random_normal( [ 3, 3, 64, 128 ], stddev = 0.1)),
    'wd1' : tf.Variable(tf.random_normal( [ 7*7*128 , 1024 ], stddev = 0.1)),
    'wd2' : tf.Variable(tf.random_normal( [ 1024, n_output ] , stddev = 0.1))
}

biases = {
    
    'bc1' : tf.Variable(tf.random_normal( [ 64 ] , stddev = 0.1)),
    'bc2' : tf.Variable(tf.random_normal( [128] , stddev = 0.1)),
    'bd1' : tf.Variable(tf.random_normal( [1024] , stddev = 0.1)),
    'bd2' : tf.Variable(tf.random_normal( [n_output], stddev =0.1))
}

def conv_basic ( _input , _w , _b , _keepratio ) :
    
    #input
    _input_r = tf.reshape(_input , shape = [-1, 28, 28 , 1] )
    
    #Conv1
    _conv1 = tf.nn.conv2d( _input_r , _w['wc1'] , strides = [ 1, 1, 1, 1] ,  padding = 'SAME' )
    _conv1 = tf.nn.batch_normalization( _conv1 ,  0.001 , 1.0 , 0, 1 , 0.0001 )  #  ?
    _conv1 = tf.nn.relu(tf.nn.bias_add( _conv1 , _b['bc1'] ))  
    _pool1 = tf.nn.max_pool(_conv1 , ksize = [1, 2, 2, 1] , strides = [1, 2, 2, 1] , padding = 'SAME')
    _pool1_dp1 = tf.nn.dropout( _pool1 , _keepratio)
    
    # Conv2
    _conv2 = tf.nn.conv2d( _pool1_dp1 , _w['wc2'] , strides = [ 1, 1, 1, 1] , padding = 'SAME')
    _conv2 = tf.nn.batch_normalization(_conv2 , 0.001 , 1.0, 0 , 1, 0.0001)
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2 , _b['bc2'] ))
    _pool2 = tf.nn.max_pool(_conv2 , ksize = [1, 2, 2, 1], strides = [ 1, 2, 2, 1],  padding ='SAME' )
    _pool_dp2 = tf.nn.dropout( _pool2 , _keepratio)
    
    #Vectorize
    _dense1 = tf.reshape( _pool_dp2 , [-1 , _w['wd1'].get_shape().as_list()[ 0 ] ] )
    
    # Fc1
    _fc1 = tf.nn.relu( tf.add( tf.matmul( _dense1 , _w[ 'wd1' ] ) ,  _b[ 'bd1' ] ) )
    _fc_dp1 = tf.nn.dropout( _fc1 , _keepratio )
    
    #Fc2
    _out = tf.add( tf.matmul( _fc_dp1 , _w[ 'wd2' ] ) , _b[ 'bd2' ] )
    
    # Return everything
    out = {
        'input_r' : _input_r,
        'conv1' : _conv1,
        'pool1' : _pool1,
        'conv2' : _conv2,
        'pool2' : _pool2,
        'pool_dp2' : _pool_dp2,
        'dense' : _dense1,
        'fc1' : _fc1,
        'fc_dp1' : _fc_dp1,
        'out' : _out
    }
    
    return out

In [24]:
# tf  Graph  input
x = tf.placeholder( tf.float32, [ None , n_input] )
y = tf.placeholder( tf.float32, [ None , n_output] )

# Parameters
learning_rate       = 0.001
training_epochs   = 5
batch_size            = 100
display_step       = 1

# Functions ! 333
_pred = conv_basic( x , weights , biases, _keepratio ) [ 'out' ]

cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( logits = _pred , labels = y ) )   #   TF1.0 bug  fixed  必须加等式

optm = tf.train.AdamOptimizer( learning_rate = learning_rate ).minimize( cost ) 

_corr = tf.equal(tf.argmax( _pred , 1) , tf.argmax( y , 1 ) )  # count  corrects

accr = tf.reduce_mean( tf.cast( _corr , tf.float32) )  #  Accuracy

init = tf.global_variables_initializer()

# Saver
save_step = 1;

savedir = " nets/ "

saver = tf.train.Saver( max_to_keep = training_epochs )

print ( " Network  ready  to  Go ! " )

 Network  ready  to  Go ! 


In [ ]:
do_train = 1

sess = tf.Session()
sess.run(init)

if do_train == 1:
    for epoch in range ( training_epochs) : 
            avg_cost = 0.
            total_batch = int ( mnist.train.num_examples/batch_size)
            
            # 循环 all  batches
            for i in range ( total_batch ) :
                batch_xs , batch_ys = mnist.train.next_batch( batch_size )
                
                # fit training using batch data
                sess.run( optm , feed_dict = { x: batch_xs , y: batch_ys} ) 
                
                # compute average loss 
                avg_cost += sess.run( cost , feed_dict = { x: batch_xs , y: batch_ys}) / total_batch
                
            # Display logs per epoch step
            if epoch % display_step == 0 :
                print ( " Epoch :  %4d / %4d  cost : %.9f "  %  (epoch , training_epochs , avg_cost ) )
                
                train_acc  = sess.run( accr , feed_dict = { x: batch_xs ,  y: batch_ys} )
                
                print ( " Training accuracy : %.3f "  %  ( train_acc ) )
                
                test_acc = sess.run( accr , feed_dict = { x: testimg ,  y: testlabel} )
                
                print ( " Test accuracy :  %.3f "   %  ( test_acc ) )
                
            # Save Net
        #   if epoch  %  save_step == 0 :
           #      saver.save( sess , " nets/cnn_sample.ckpt - " + str(epoch) )
        
    print ( " Optimization finished ! ")

 Epoch :     0 /    5  cost : 0.529632958 
 Training accuracy : 0.960 
 Test accuracy :  0.971 
 Epoch :     1 /    5  cost : 0.080203810 
 Training accuracy : 0.980 
 Test accuracy :  0.977 
 Epoch :     2 /    5  cost : 0.055751287 
 Training accuracy : 0.970 
 Test accuracy :  0.983 
 Epoch :     3 /    5  cost : 0.042000231 
 Training accuracy : 0.950 
 Test accuracy :  0.983 
 Epoch :     4 /    5  cost : 0.034283692 
 Training accuracy : 0.990 
 Test accuracy :  0.987 
 Optimization finished ! 
